In [ ]:
import numpy as np
import torch
import torchvision
from torch import nn
import matplotlib
from matplotlib import pyplot as plt
import tqdm.notebook as tqdm

In [ ]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

np_array = np.array(data)
x_np = torch.from_numpy(np_array)

shape = (2, 3,)
rand_tensor = torch.rand(shape)
np_rand_array = rand_tensor.numpy()

print(f"Tensor from np: \n {x_np} \n")
print(f"Rand Tensor: \n {rand_tensor} \n")
print(f"Rand Numpy Array: \n {np_rand_array} \n")

In [ ]:
tensor = torch.ones(4, 4)
print('First row: ', tensor[0])
print('First column: ', tensor[:, 0])

y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

scalar = torch.sum(y1)
item = scalar.item()
print("Sum as a tensor:", scalar, ", Sum as an item:", item)

In [ ]:
x = torch.ones(5)
y = torch.zeros(1)
w = torch.randn(5, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)
pred = torch.sigmoid(torch.matmul(x, w) + b)
loss = torch.nn.functional.binary_cross_entropy(pred, y)
loss.backward()
print("W gradient:", w.grad)
print("b gradient:", b.grad)

optimizer = torch.optim.SGD([w, b], lr=0.1)
print("Weight before", w)
optimizer.step()
print("Updated weight", w)

with torch.no_grad():
    # ...

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)
tensor = torch.zeros(4,4, requires_grad=True)
tensor = tensor.to(device)

tensor = 10*(tensor + 1)

tensor = tensor.detach().cpu()

tensor_np = tensor.numpy()

In [ ]:
class Perception(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.layer = nn.Linear(in_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.layer(x)).squeeze(-1)

perceptron = Perceptron(10)
perceptron = perceptron.to(device)
print("Parameters", list(perceptron.parameters()))

In [ ]:
c1_x1, c1_x2 = np.random.multivariate_normal([-2.5,3], [[1, 0.3],[0.3, 1]], 100).T
c2_x1, c2_x2 = np.random.multivariate_normal([1,1], [[2, 1],[1, 2]], 100).T
c1_X = np.vstack((c1_x1, c1_x2)).T
c2_X = np.vstack((c2_x1, c2_x2)).T
train_X = np.concatenate((c1_X, c2_X))
train_y = np.concatenate((np.zeros(100), np.ones(100)))
permutation = np.random.permutation(train_X.shape[0])
train_X = train_X[permutation]
train_y = train_y[permutation]

plt.plot(c1_x1, c1_x2, 'x')
plt.plot(c2_x1, c2_x2, 'o')
plt.axis('equal')
plt.show()

In [ ]:
dataset = torch.utils.data.TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle = True)

for x, y in dataloader:
  print("Batch x:", x)
  print("Batch y:", y)
  break

del dataloader

In [ ]:
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle = False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)

epochs = 10
batch_size = 10
learning_rate = 0.01

num_features = dataset[0][0].shape[0]
model = Perceptron(num_features).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = torch.nn.BCELoss()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle = True)

mode.train()
for epoch in range(epochs):
  training_losses = []
  for x, y in tqdm.tqdm(dataloader, unit="batch"):
    x, y = x.float().to(device), y.float().to(device)
    optimizer.zero_grad()
    pred = model(x)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()
    training_losses.append(loss.item())
  print("Finished Epoch", epoch+1, ",training loss: ", np.mean(training_losses))

with torch.no_grad():
  model.eval()
  num_correct = 0
  for x, y in dataloader:
    x,y = x.float().to(device), y.float().to(device)
    pred = model(x)
    num_correct += torch.sum(torch.round(pred) == y).item()
  print("Final Accuracy:", num_correct / len(dataset))
  model.train()

In [ ]:
transform = torchvision.transforms.ToTensor()

training_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

validation_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

In [ ]:
images = [training_data[i][0] for i in range(9)]
plt.imshow(torchvision.utils.make_grid(torch.stack(images), nrow=3, padding=5).numpy().transpose((1, 2, 0)))

In [ ]:
transform = torchvision.transforms.ToTensor()

training_data = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)

In [ ]:
images = [training_data[i][0] for i in range(9)]
plt.imshow(torchvision.utils.make_grid(torch.stack(images), nrow=3, padding=5).numpy().transpose((1, 2, 0)))

In [ ]:
from PIL import Image
import os

class CIFAR10Test(torchvision.datasets.VisionDataset):
    def __init__(self, transform = None, target_transform = None):
        super(CIFAR10Test, self).__init__(None, transform=transform,
                                          target_transform=target_transform)
        assert os.path.exists("cifar10_test_data.npy"), "You must upload the test data to the file system."
        self.data = [np.load("cifar10_test_data.npy", allow_pickle=False)]

        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
        self.data = self.data.transpose((0,2,3,1))

    def __getitem__(self, index):
        img, _ = super(CIFAR10Test, self).__getitem__(index)
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __len__(self) -> int:
        return len(self.data)

sting_data = CIFAR10Test(
    transform=transform,
)

In [ ]:
# ...

test_dataloader = torch.utils.data.DataLoader()

predictions = None

In [ ]:
for x, y in test_data:
  # ...

for x in test_data:
  # ...

In [ ]:
import pandas as pd

if isinstance(predictions, np.ndarray):
  predictions = predictions.astype(int)
else:
  predictions = np.array(predictions, dtype = int)
assert predictions.shape == (len(testing_data),), "Predictions were not the correct shape"
df = pd.DataFrame({'Category': predictions})
df.index += 1
df.to_csv('predictions.csv', index_label='Id')
